In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp

In [2]:
training_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')

print(training_data.shape)

(42000, 785)


In [3]:
training_data.head(10)

label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   
5      0       0       0       0       0       0       0       0       0   
6      7       0       0       0       0       0       0       0       0   
7      3       0       0       0       0       0       0       0       0   
8      5       0       0       0       0       0       0       0       0   
9      3       0       0       0       0       0       0       0       0   

   pixel8    ...     pixel774  pixel775  pixel776  pixel777  pixel778  \
0       0    ...            0         0         0         0         0   
1       0    ...            0         0         0         0         0   
2       0    ...            0         0         0         0         0   
3       0    ...            0         0         0         0         0   
4       0    ...            0         0         0         0         0   
5       0    ...            0         0         0         0         0   
6       0    ...            0         0         0         0         0   
7       0    ...            0         0         0         0         0   
8       0    ...            0         0         0         0         0   
9       0    ...            0         0         0         0         0   

   pixel779  pixel780  pixel781  pixel782  pixel783  
0         0         0         0         0         0  
1         0         0         0         0         0  
2         0         0         0         0         0  
3         0         0         0         0         0  
4         0         0         0         0         0  
5         0         0         0         0         0  
6         0         0         0         0         0  
7         0         0         0         0         0  
8         0         0         0         0         0  
9         0         0         0         0         0  

[10 rows x 785 columns]

In [4]:
X = np.array(training_data.drop(['label'], axis = 1))
Y = np.array(training_data['label'])
X.shape, Y.shape

((42000, 784), (42000,))

In [5]:
# scaled
X = X / 255.0
X_test = np.array(test_data) / 255.0

X_test.shape

(28000, 784)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2)

In [9]:
X_train.shape, X_val.shape

((33600, 784), (8400, 784))

In [10]:
Y_train.shape, Y_val.shape

((33600,), (8400,))

In [8]:
print('\ntrain_X: min = %.3f, max = %.3f' %(X_train.min(), X_train.max()))
print('train_Y: min = %d, max = %d' %(Y_train.min(), Y_train.max()))


train_X: min = 0.000, max = 1.000
train_Y: min = 0, max = 9


### training

In [11]:
from sklearn.svm import SVC
clf = SVC(C = 10, gamma = 0.01)
%time clf.fit(X_train, Y_train)

#%time clf.score(X_val, Y_val)

Wall time: 3min 21s


SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [12]:
%time Y_val_pred = clf.predict(X_val)

In [17]:
accuracy = np.mean(Y_val == Y_val_pred)
accuracy

0.9779761904761904

In [18]:
%time Y_train_pred = clf.predict(X_train)

Wall time: 5min 34s


In [19]:
accuracy_train = np.mean(Y_train == Y_train_pred)
accuracy_train

0.9996130952380953

In [20]:
%time Y_test_pred = clf.predict(X_test)

Wall time: 5min 36s


In [22]:
Y_test_pred

array([2, 0, 9, ..., 3, 9, 2], dtype=int64)

In [24]:
Id = np.arange(1,28001)

In [25]:
Id

array([    1,     2,     3, ..., 27998, 27999, 28000])

In [29]:
sub = {'ImageId': Id,
       'Label': Y_test_pred}
df = pd.DataFrame(sub, columns = ['ImageId', 'Label'])

In [30]:
df.to_csv('./Submission.csv', index=False)

### Training 1

In [45]:
def runSVC(C_, gamma_, name):
    clf = SVC(C = C_, gamma = gamma_)
    print('Training ...')
    %time clf.fit(X_train, Y_train)
    print('Predicting valdiadtion ...')
    %time Y_val_pred = clf.predict(X_val)
    accuracy_val = np.mean(Y_val == Y_val_pred)
    print('val accuracy: ', accuracy_val)
    print('Predicting test ...')
    %time Y_test_pred = clf.predict(X_test)
    sub = {'ImageId': Id,
       'Label': Y_test_pred}
    df = pd.DataFrame(sub, columns = ['ImageId', 'Label'])
    path = './' + name
    df.to_csv(path, index=False)

In [46]:
%time Y_pred = runSVC(100, 0.1, 'Submission_3.csv')

Wall time: 52min 49s
Wall time: 5min 38s
Wall time: 17min 25s
Wall time: 1h 16min 1s


### Training 2

In [36]:
clf_1 = SVC(C = 100, gamma = 0.01)
%time clf_1.fit(X, Y)

Wall time: 4min 5s


SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [37]:
%time clf_1.score(X, Y)

Wall time: 7min 28s


1.0

In [38]:
%time Y_test_pred_1 = clf_1.predict(X_test)

Wall time: 5min 57s


In [39]:
sub = {'ImageId': Id,
       'Label': Y_test_pred_1}
df_1 = pd.DataFrame(sub, columns = ['ImageId', 'Label'])

In [40]:
df_1.to_csv('./Submission_2.csv', index=False)